#### Installing Dependencies

In [32]:
#importing dependencies
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


#### Imports

In [33]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import time

#### Mediapipe hand model initialization

In [34]:
#Draws line segments
mp_drawing = mp.solutions.drawing_utils
#mp hands model
mp_hands = mp.solutions.hands

#### Note - Remember to insert landmarks.png into markdown here for reference

![alt text] (https://github.com/NamanBiyani06/HandPoseDetection/blob/main/images/hand_landmarks.png?raw=true)

#### Associating each landmark with its index

In [35]:
WRIST = 0
THUMB_CMC = 1
THUMB_MCP = 2
THUMB_IP = 3
THUMB_TIP = 4
INDEX_FINGER_MCP = 5
INDEX_FINGER_PIP = 6
INDEX_FINGER_TIP = 8
MIDDLE_FINGER_MCP = 9
MIDDLE_FINGER_PIP = 10
MIDDLE_FINGER_TIP = 12
RING_FINGER_MCP = 13
RING_FINGER_PIP = 14
RING_FINGER_TIP = 16
PINKY_MCP = 17
PINKY_PIP = 18
PINKY_TIP = 20

#### Functions to dectect when a hand gesture is performed

##### Functions return gesture name depending if joint positions match the generalized positions of the specific gesture

In [36]:
# Is the model performing a fist
def fist_gesture(points):
    if points[MIDDLE_FINGER_MCP].y < points[MIDDLE_FINGER_TIP].y \
    and points[PINKY_MCP].y < points[PINKY_TIP].y \
    and points[RING_FINGER_MCP].y < points[RING_FINGER_TIP].y:
        return "Fist"


#### Opening a Computer Vision window and creating a loop to update the model each frame

In [37]:
#getting webcam feed #0
cap = cv2.VideoCapture(0)

cv2.startWindowThread()

#initializing gesture to prevent errors
gesture = "init"

#declaring following as "hands"
#detection threshold for intitial detection to be successful
#tracking threshold for tracking after initial detection
with mp_hands.Hands(min_detection_confidence = 0.8, min_tracking_confidence = 0.5) as hands:
    #reading through each frame
    while cap.isOpened():
        ret,frame = cap.read()

        #detections
        #recolours from BGR to RGB
        #passes in frame, outputs image and RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #image is flipped on the horizontal
        image = cv2.flip(image, 1)

        image.flags.writeable = False
        #detections
        results = hands.process(image)
        image.flags.writeable = True
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #checking if we have results
        if results.multi_hand_landmarks:
            #putting landmarks into landmarks
            for landmark in results.multi_hand_landmarks:
                landmarks = landmark.landmark
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius = 4), mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))

            
            #Obtaining current gesture
            if fist_gesture(landmarks):
                gesture = fist_gesture(landmarks)
            else:
                gesture = "NULL"

        
        # font used while displaying text
        font = cv2.FONT_HERSHEY_SIMPLEX
        # putting the current gesture on the window
        cv2.putText(image, gesture, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)

        cv2.imshow('Hand Tracking', image)
        cv2.setWindowProperty('Hand Tracking', cv2.WND_PROP_TOPMOST, 1)

        

        #close window if q is pressed
        if cv2.waitKey(10) & 0xff == ord('q'):
            cv2.waitKey(1)
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break

cv2.destroyAllWindows()
cap.release()

